First Attempt at a Simple and Content Based Movie Recommender System using a dataset from

https://www.kaggle.com/stefanoleone992/rotten-tomatoes-movies-and-critics-datasets/download

The following are the steps involved:

- Decide on the metric or score to rate movies on.
- Calculate the score for every movie.
- Sort the movies based on the score and output the top results.

In [0]:
import pandas as pd

#Load data
metadata = pd.read_csv('../movies_dsc/rottentomatoes/data/rotten_tomatoes_movies.csv', low_memory=False)

In [0]:
# v is the number of votes for the movie;
# m is the minimum votes required to be listed in the chart;
# R is the average rating of the movie; And
# C is the mean vote across the whole report

C = metadata['audience_rating'].mean()
# print(C)
m = metadata['audience_count'].quantile(0.90)
# print(m)
q_movies = metadata.copy().loc[metadata['audience_count'] >= m]
# q_movies.shape




In [0]:
def weighted_rating(x, m=m, C=C):
    v = x['audience_count']
    R = x['audience_rating']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)
#Print the top 15 movies
# q_movies[['movie_title', 'audience_count', 'audience_rating', 'score']].head(15)

In [0]:
#Print plot overviews of the first 5 movies.


#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
metadata['movie_info'] = metadata['movie_info'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata['movie_info'])

#Output the shape of tfidf_matrix
# tfidf.shape

In [0]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#Construct a reverse map of indices and movie titles
indices = pd.Series(metadata.index, index=metadata['movie_title']).drop_duplicates()

# indices.shape

In [0]:
def get_list(x):
  temp = x
  if str(x).count(",") > 2:
   temp = x.split(",")
   print(temp)
   temp = temp[0] + ',' + temp[1] + ',' + temp[2]
   print(temp)
  return temp


In [0]:
features = ['cast', 'genre']
for feature in features:
    # print("FEATURE: ",feature, " ", metadata[feature], " || ",metadata['cast'])
    metadata[feature] = metadata[feature].apply(get_list)

In [0]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [0]:
features = ['cast', 'directors', 'genre']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

def create_soup(x):
    return ' '.join(x['cast']) + ' ' + x['directors'] + ' ' + ' '.join(x['genre'])
# Create a new soup feature
metadata['soup'] = metadata.apply(create_soup, axis=1)


In [0]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
# Reset index of your main DataFrame and construct reverse mapping as before
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['movie_title'])

In [0]:

# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the n most similar movies
    sim_scores = sim_scores[1:51]
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    #randomize order of titles so different output is generated
    import  random
    random.shuffle(movie_indices)
    #cut newly ordered list down to 10 from n 
    movie_indices = movie_indices[:len(movie_indices)-40]
    # Return the top 10 most similar movies
    return metadata[['movie_title', 'in_theaters_date', 'audience_rating', 'tomatometer_rating', 'cast']].iloc[movie_indices]

In [0]:
get_recommendations('The Dark Knight Rises', cosine_sim2)